In [15]:
from phishingDetection.urlPishDect.modules.Prediction import PredictionURLS
from phishingDetection.emailPishDect.modules.FeatureExtractionEmail import FeatureExtractionEmail
from flask_cors import CORS
from flask import Flask,render_template, request,jsonify

import warnings
import joblib

Email 

In [16]:
def loadPredict(predict):
    
    loaded_model = joblib.load('phishingDetection/emailPishDect/model/best_model.pkl')
    label = loaded_model.predict(predict)

    return label


def predictionEmail(email_content):
        
    obj=FeatureExtractionEmail(email_content)
    df = obj.df
    prad = loadPredict(df)
    
    return prad

Urls

In [17]:
def predictionURLS( urls=[]):
    
    warnings.filterwarnings("ignore", category=UserWarning, message="Trying to unpickle estimator.*from version.*when using version.*")  

    obj = PredictionURLS(urls)

    return obj.resultOutput


In [18]:

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes


For Website API Calls

In [19]:
@app.route('/')
def home():
    return render_template('index.html')


@app.route('/urlpredict', methods=['POST'])
def urlpredict():
    
    url = request.form.get('url', '')
    
    url_result = predictionURLS([url])

    # Render the result template with the prediction result
    return render_template('index.html', url_result=url_result)




@app.route('/emailpredict', methods=['POST'])
def emailpredict():
    
    email = request.form.get('email', '')
    
    email_result = predictionEmail(email)

    # Render the result template with the prediction result
    return render_template('index.html', email_result=email_result)


For Extention API Calls

In [20]:


@app.route('/urlpredictExt', methods=['POST'])
def urlpredictExt():
    
    data = request.json
    url = data.get('url', '')
    
    url_result = predictionURLS([url])

    result = 'Phishing'
    if(url_result[9] == '0'):
        result = 'Safe'

    return {"result": f"Current Url Result: {result}"}

@app.route('/emailpredictExt', methods=['POST'])
def emailpredictExt():
    
    data = request.json
    email = data.get('email', '')
    

    email_result = predictionEmail(email)
    
    print(email_result)

    result = 'Phishing'
    if(email_result[0] == 0):
        result = 'Safe'
    # Render the result template with the prediction result
    return {"result": f"Email Result: {result}"}



In [21]:

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.20.193.211:8080
Press CTRL+C to quit
172.20.193.211 - - [11/Mar/2024 11:47:48] "GET / HTTP/1.1" 200 -
172.20.193.211 - - [11/Mar/2024 11:47:48] "GET /static/css/styles.css HTTP/1.1" 200 -
172.20.193.211 - - [11/Mar/2024 11:47:48] "GET /favicon.ico HTTP/1.1" 404 -
172.20.193.211 - - [11/Mar/2024 11:48:10] "OPTIONS /urlpredictExt HTTP/1.1" 200 -
172.20.193.211 - - [11/Mar/2024 11:48:20] "POST /urlpredictExt HTTP/1.1" 200 -
172.20.193.211 - - [11/Mar/2024 11:48:57] "OPTIONS /urlpredictExt HTTP/1.1" 200 -
172.20.193.211 - - [11/Mar/2024 11:49:01] "POST /urlpredictExt HTTP/1.1" 200 -
172.20.193.211 - - [11/Mar/2024 11:49:02] "OPTIONS /urlpredictExt HTTP/1.1" 200 -
172.20.193.211 - - [11/Mar/2024 11:49:06] "POST /urlpredictExt HTTP/1.1" 200 -
172.20.193.211 - - [11/Mar/2024 11:50:18] "OPTIONS /urlpredictExt HTTP/1.1" 200 -
172.20.193.211 - - [11/Mar/2024 11:50:22] "POST /urlpredictExt HTTP/1.1"